In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#from sklearn.tree import DecisionTreeRegressor
#from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
#from sklearn.tree import plot_tree
from sklearn.preprocessing import StandardScaler
%matplotlib inline
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix
from sklearn.metrics import plot_confusion_matrix
pd.set_option('display.max_columns', None)
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import TomekLinks
import warnings
warnings.filterwarnings('ignore')

In [2]:
churnData = pd.read_csv("DATA_Customer-Churn.csv")

In [3]:
churnData.head(5)

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,MonthlyCharges,TotalCharges,Churn
0,Female,0,Yes,No,1,No,No,Yes,No,No,No,No,Month-to-month,29.85,29.85,No
1,Male,0,No,No,34,Yes,Yes,No,Yes,No,No,No,One year,56.95,1889.5,No
2,Male,0,No,No,2,Yes,Yes,Yes,No,No,No,No,Month-to-month,53.85,108.15,Yes
3,Male,0,No,No,45,No,Yes,No,Yes,Yes,No,No,One year,42.30,1840.75,No
4,Female,0,No,No,2,Yes,No,No,No,No,No,No,Month-to-month,70.70,151.65,Yes


In [4]:
churnData.dtypes

gender               object
SeniorCitizen         int64
Partner              object
Dependents           object
tenure                int64
PhoneService         object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
Contract             object
MonthlyCharges      float64
TotalCharges         object
Churn                object
dtype: object

### Data Cleaning

In [5]:
churnData.TotalCharges = pd.to_numeric(churnData.TotalCharges, errors="coerce")

In [7]:
churnData.isnull().sum() #Checking null values

gender               0
SeniorCitizen        0
Partner              0
Dependents           0
tenure               0
PhoneService         0
OnlineSecurity       0
OnlineBackup         0
DeviceProtection     0
TechSupport          0
StreamingTV          0
StreamingMovies      0
Contract             0
MonthlyCharges       0
TotalCharges        11
Churn                0
dtype: int64

In [8]:
churnData.TotalCharges.fillna(churnData.TotalCharges.mean(),inplace=True)

### Data Modellling

In [9]:
# features: tenure, SeniorCitizen, MonthlyCharges and TotalCharges:
# Scaling the features
# Split the data
# Finding appropiate regression and checking its accuracy

In [10]:
# Changing data to numerical
x = churnData[["tenure", "SeniorCitizen", "MonthlyCharges", "TotalCharges"]]
churnData.Churn= churnData.Churn.replace("No", 0).replace("Yes", 1)
y = churnData.Churn

In [14]:
Data = pd.concat([x, y], axis=1)

In [15]:
Data.head(10)

,tenure,SeniorCitizen,MonthlyCharges,TotalCharges,Churn
0,1,0,29.85,29.85,0
1,34,0,56.95,1889.50,0
2,2,0,53.85,108.15,1
3,45,0,42.30,1840.75,0
4,2,0,70.70,151.65,1
5,8,0,99.65,820.50,1
6,22,0,89.10,1949.40,0
7,10,0,29.75,301.90,0
8,28,0,104.80,3046.05,1
9,62,0,56.15,3487.95,0


In [16]:
def scaling_data(df):

    #standardizing
    scaler = StandardScaler()
    scaler.fit(df)  
    x = scaler.transform(df) 
    x = pd.DataFrame(x, columns = df.columns) 

In [17]:
scaling_data(x)

In [18]:
# Splitting function
def fitting_model(x,y):
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state = 10)
    
    model = LogisticRegression()
    model.fit(x_train, y_train)
            
    predicted_y = model.predict(x_test)
    predicted_y_train = model.predict(x_train)

In [19]:
fitting_model(x,y)

### Handling imbalance data

In [23]:
churnData['Churn'].value_counts()

0    5174
1    1869
Name: Churn, dtype: int64

In [24]:
category_0 = churnData[churnData['Churn'] == 0]
category_1 = churnData[churnData['Churn'] == 1]

category_0 = category_0.sample(len(category_1))
print(category_0.shape)
print(category_1.shape)

(1869, 16)
(1869, 16)


In [25]:
data = pd.concat([category_0, category_1], axis=0)
data = data.sample(frac=1)
data['Churn'].value_counts()

0    1869
1    1869
Name: Churn, dtype: int64

### Upsampling using SMOTE

In [27]:
smote = SMOTE()

In [30]:
display(x.head(5))

display(y.head(5))

,tenure,SeniorCitizen,MonthlyCharges,TotalCharges
0,1,0,29.85,29.85
1,34,0,56.95,1889.50
2,2,0,53.85,108.15
3,45,0,42.30,1840.75
4,2,0,70.70,151.65


0    0
1    0
2    1
3    0
4    1
Name: Churn, dtype: int64

In [29]:
x_sm, y_sm = smote.fit_resample(x, y)
y_sm.value_counts()

0    5174
1    5174
Name: Churn, dtype: int64

### Downsampling using tomklinks

In [34]:
# It does not make the two classes equal but only removes the points from the majority class that are close to other points in minority class.
tl = TomekLinks('majority')
x_tl, y_tl = tl.fit_resample(x, y)
y_tl.value_counts()

0    4620
1    1869
Name: Churn, dtype: int64